In [1]:
%config IPCompleter.greedy = True

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import random
import pprint

import nltk
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

from collections import defaultdict

### 1. Load NLTK and Test Dataset</font>
***

#### Loading Treebank tagged sentences using **universal** tagset

`VERB - verbs (all tenses and modes)
NOUN - nouns (common and proper)
PRON - pronouns 
ADJ - adjectives
ADV - adverbs
ADP - adpositions (prepositions and postpositions)
CONJ - conjunctions
DET - determiners
NUM - cardinal numbers
PRT - particles or other function words
X - other: foreign words, typos, abbreviations
. - punctuation
`

In [4]:
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [5]:
# observe a few tagged sentences from the corpora
print(nltk_data[:2])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


#### Loading Test Data

In [6]:
file_object = open(r"test-sentences.txt","r", encoding="latin1")
test_data = file_object.read()
test_data

"Android is a mobile operating system developed by Google.\nAndroid has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.\nGoogle and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.\nTwitter is an online news and social networking service on which users post and interact with messages known as tweets.\nBefore entering politics, Donald Trump was a domineering businessman and a television personality.\nThe 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.\nThis is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.\nShow me the cheapest round trips from Dallas to Atlanta\nI would like to see flights from Denver to Philadelphia.\nShow me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.\nNASA invited social media users to experience the launch of ICESAT-2 Satell

In [7]:
# number of words in the test dataset
test_data_words = nltk.word_tokenize(test_data)
len(test_data_words)

181

#### Tagging Test Dataset With NLTK POS Tagger

In [8]:
test_tagged_words = {}
test_tagged = nltk.pos_tag(test_data_words, tagset='universal')
universal_tagset = [
    'VERB', 'NOUN', 'PRON', 'ADJ', 'ADV', 
    'ADP', 'CONJ', 'DET', 'NUM', 'PRT', 'X', '.'
]

for utag in universal_tagset:
    test_tagged_words[utag] = sorted(
        set([word for (word, tag) in test_tagged if tag == utag]))

i = random.randrange(len(universal_tagset))

pprint.pprint('words with tagged with {}'.format(universal_tagset[i]))
pprint.pprint(test_tagged_words[universal_tagset[i]])

'words with tagged with .'
[',', '.']


### 2. Split data into train and validation datasets
***

In [9]:
train_set, validation_set = train_test_split(nltk_data,
                                             test_size=0.05,
                                             random_state=1234)

print('Number of sentences in train dataset : {0}'.format(len(train_set)))
print('Number of sentences in validation dataset : {0}'.format(len(validation_set)))

Number of sentences in train dataset : 3718
Number of sentences in validation dataset : 196


In [10]:
train_tagged_words = [tup for sent in train_set for tup in sent]

In [11]:
tokens = [pair[0] for pair in train_tagged_words]
print('total number of words in the training set : {0}'.format(len(tokens)))

vocabulary = set(tokens)
print('total number of unique words in the training set: {0}'.format(len(vocabulary)))

total number of words in the training set : 95799
total number of unique words in the training set: 12073


In [12]:
all_tags = [pair[1] for pair in train_tagged_words]
unique_tags = sorted(set(all_tags))

print('number of tags in the universal tagset : {}'.format(len(unique_tags)))
print(unique_tags)

number of tags in the universal tagset : 12
['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


### 3. Helper Functions
***

#### Store number of times a tag 'T' appears in the training dataset

In [13]:
tag_count_dict = dict()

for utag in unique_tags:
    tag_list = [pair[1] for pair in train_tagged_words if pair[1] == utag]
    tag_count_dict[utag] = len(tag_list)
    
print(tag_count_dict)    

{'.': 11130, 'ADJ': 6063, 'ADP': 9387, 'ADV': 3052, 'CONJ': 2144, 'DET': 8269, 'NOUN': 27471, 'NUM': 3364, 'PRON': 2619, 'PRT': 3070, 'VERB': 12910, 'X': 6320}


#### List of Unknown Words in Validation Dataset

In [14]:
val_data_unknown_words = [word for sent in validation_set for (word, tag) in sent if word not in vocabulary]
print('number of unknown words in validation data set : {0}'.format(len(set(val_data_unknown_words))))

number of unknown words in validation data set : 335


#### List of Unknown Words in Test Dataset

In [15]:
test_data_unknown_words = [word for word in test_data_words if word not in vocabulary]
print('number of unknown words in test data set : {0}'.format(len(set(test_data_unknown_words))))

number of unknown words in test data set : 28


#### Calculate Number of Words correctly tagged in Test Dataset

In [16]:
def calc_test_dataset_accuracy(tagged_test_set):
    total_words = 0
    correct_tagged_words = 0

    for word, tag in tagged_test_set:
        try:
            list_for_tag = test_tagged_words[tag]
        except KeyError:
            list_for_tag = []

        total_words += 1

        if word in list_for_tag:
            correct_tagged_words += 1

    print('total words - {0}. correctly tagged words - {1}. accuracy - {2}'.
          format(total_words, correct_tagged_words,
                 correct_tagged_words / total_words))

### 4. Learning HMM Model Parameters
***

#### Emission Probabilities

In [17]:
def word_given_tag(word, tag, train_bag=train_tagged_words):

    w_given_tag_list = [
        pair[0] for pair in train_bag if pair[0] == word and pair[1] == tag
    ]
    count_w_given_tag = len(w_given_tag_list)

    return count_w_given_tag

#### Transition Probabilities

In [18]:
def t2_given_t1(t2, t1, train_bag=train_tagged_words):
    
    count_t2_t1 = 0

    for index in range(len(all_tags) - 1):
        if all_tags[index] == t1 and all_tags[index + 1] == t2:
            count_t2_t1 += 1

    return count_t2_t1

In [19]:
tags_matrix = np.zeros((len(unique_tags), len(unique_tags)), dtype='float32')

for i, t1 in enumerate(list(unique_tags)):
    for j, t2 in enumerate(list(unique_tags)):
        count_t1 = tag_count_dict[t1]
        tags_matrix[i, j] = t2_given_t1(t2, t1) / count_t1

In [20]:
df_tag = pd.DataFrame(tags_matrix,
                      columns=list(unique_tags),
                      index=list(unique_tags))

df_tag

.       ADJ       ADP       ADV      CONJ       DET      NOUN  \
.     0.094070  0.044654  0.090386  0.051932  0.057772  0.173226  0.223091   
ADJ   0.065809  0.065314  0.077519  0.004948  0.016658  0.004948  0.698499   
ADP   0.039842  0.105785  0.016512  0.013849  0.000959  0.322893  0.322893   
ADV   0.134666  0.129751  0.118611  0.081258  0.006881  0.068480  0.031127   
CONJ  0.033116  0.118937  0.052705  0.055970  0.000466  0.118470  0.348881   
DET   0.017777  0.203652  0.009191  0.012698  0.000484  0.005442  0.638650   
NOUN  0.239307  0.012231  0.177023  0.017182  0.042263  0.013250  0.264898   
NUM   0.115933  0.032402  0.035672  0.002973  0.013377  0.002973  0.354637   
PRON  0.040473  0.073692  0.023291  0.032837  0.004582  0.009164  0.207331   
PRT   0.041694  0.084039  0.021173  0.009772  0.002280  0.099674  0.247883   
VERB  0.035167  0.065221  0.091402  0.083501  0.005577  0.133617  0.110844   
X     0.162816  0.016456  0.144937  0.026108  0.010759  0.054114  0.062184   

           NUM      PRON       PRT      VERB         X  
.     0.080593  0.065768  0.002336  0.088769  0.027314  
ADJ   0.021112  0.000660  0.010886  0.012205  0.021442  
ADP   0.062001  0.070203  0.001491  0.008522  0.035048  
ADV   0.031455  0.015400  0.014744  0.344364  0.023263  
CONJ  0.041511  0.057369  0.005131  0.158582  0.008862  
DET   0.022373  0.003749  0.000242  0.039545  0.046197  
NOUN  0.009537  0.004769  0.043974  0.146336  0.029231  
NUM   0.184899  0.001486  0.027051  0.018133  0.210464  
PRON  0.007255  0.007637  0.011837  0.487972  0.093929  
PRT   0.056678  0.017915  0.001954  0.402932  0.014007  
VERB  0.022696  0.035321  0.031216  0.167622  0.217816  
X     0.002690  0.056329  0.184652  0.204114  0.074842

#### Start Probabilities

In [21]:
df_tag.loc['.', :]

.       0.094070
ADJ     0.044654
ADP     0.090386
ADV     0.051932
CONJ    0.057772
DET     0.173226
NOUN    0.223091
NUM     0.080593
PRON    0.065768
PRT     0.002336
VERB    0.088769
X       0.027314
Name: ., dtype: float32

### 5. Vanilla Viterbi Based POS Tagger
***

In [22]:
def Viterbi_Vanilla(words, train_bag=train_tagged_words):
    state = []
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in unique_tags:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                transition_p = df_tag.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag) / tag_count_dict[tag]
            state_probability = emission_p * transition_p
            
            p.append(state_probability)

        pmax = max(p)
        
        # getting state for which probability is maximum
        # tagging unknown words as 'X' to mark those as foreign words
        if pmax == 0:
            state_max = 'X'
        else:    
            state_max = unique_tags[p.index(pmax)]

        state.append(state_max)

    return list(zip(words, state))

### 6. Running Algorithm On Validation Dataset
***

In [23]:
random.seed(1234)

random_indices = [random.randint(1, len(validation_set)) for x in range(5)]

validation_run = [validation_set[i] for i in random_indices]

validation_run_base = [tup for sent in validation_run for tup in sent]

validation_untagged_words = [tup[0] for tup in validation_run_base]

In [24]:
print('number of words in selected validation set : {0}'.format(len(validation_untagged_words)))

number of words in selected validation set : 166


In [25]:
%%time

validation_tagged_sent = Viterbi_Vanilla(validation_untagged_words)

Wall time: 28.6 s


### 7. Model Validation
***

In [26]:
correct_tags = [i for i, j in zip(validation_run_base, validation_tagged_sent) if i == j]

accuracy = len(correct_tags) / len(validation_run_base)

accuracy

0.8674698795180723

In [27]:
validation_incorrect_tagged_words = [(i, j) for i, j in zip(validation_run_base, validation_tagged_sent) if i != j]

print(len(validation_incorrect_tagged_words))
validation_incorrect_tagged_words

22


[(('sell', 'NOUN'), ('sell', 'VERB')),
 (('printers', 'NOUN'), ('printers', 'X')),
 (('there', 'ADV'), ('there', 'DET')),
 (('Gunmen', 'NOUN'), ('Gunmen', 'X')),
 (('Lebanon', 'NOUN'), ('Lebanon', 'X')),
 (('assassinated', 'VERB'), ('assassinated', 'X')),
 (('Arabian', 'NOUN'), ('Arabian', 'X')),
 (('pro-Iranian', 'ADJ'), ('pro-Iranian', 'X')),
 (('Islamic', 'NOUN'), ('Islamic', 'X')),
 (('slaying', 'NOUN'), ('slaying', 'X')),
 (('avenge', 'VERB'), ('avenge', 'X')),
 (('beheading', 'NOUN'), ('beheading', 'X')),
 (('terrorists', 'NOUN'), ('terrorists', 'X')),
 (('Riyadh', 'NOUN'), ('Riyadh', 'X')),
 (('Card', 'NOUN'), ('Card', 'X')),
 (('sweepstakes', 'NOUN'), ('sweepstakes', 'X')),
 (('forthcoming', 'ADJ'), ('forthcoming', 'X')),
 (('10-year', 'NUM'), ('10-year', 'ADJ')),
 (('yen-denominated', 'ADJ'), ('yen-denominated', 'X')),
 (('about', 'ADV'), ('about', 'ADP')),
 (('redeeming', 'VERB'), ('redeeming', 'X')),
 (('convert', 'VERB'), ('convert', 'X'))]

> -  *Accuracy of Vanilla Viterbi - in **high 80s** depending on validation dataset*
-  *Number of incorrect tagged words from validation set : **22** words*

### 8. Running Algorithm On Test Dataset
***

In [28]:
%%time

test_tagged_set = Viterbi_Vanilla(test_data_words)

Wall time: 36.5 s


#### Calculate Test Dataset Accuracy

In [29]:
calc_test_dataset_accuracy(test_tagged_set)

total words - 181. correctly tagged words - 139. accuracy - 0.7679558011049724


In [30]:
test_unknown_tagged_words = [tup for tup in test_tagged_set if tup[0] in test_data_unknown_words]

print(test_unknown_tagged_words)

[('Android', 'X'), ('Google', 'X'), ('Android', 'X'), ('OS', 'X'), ('worldwide', 'X'), ('smartphones', 'X'), ('2011', 'X'), ('2013', 'X'), ('Google', 'X'), ('Twitter', 'X'), ('2015', 'X'), ('Google', 'X'), ('Twitter', 'X'), ('firehose', 'X'), ('Twitter', 'X'), ('online', 'X'), ('interact', 'X'), ('messages', 'X'), ('tweets', 'X'), ('domineering', 'X'), ('personality', 'X'), ('2018', 'X'), ('FIFA', 'X'), ('Cup', 'X'), ('21st', 'X'), ('FIFA', 'X'), ('Cup', 'X'), ('tournament', 'X'), ('contested', 'X'), ('Cup', 'X'), ('trips', 'X'), ('arriving', 'X'), ('NASA', 'X'), ('invited', 'X'), ('ICESAT-2', 'X'), ('Satellite', 'X')]


> -  *all unknown words have been assigned the <b>1st tag</b> present in the universal tagset*
-  *all unknown proper nouns like <b>Android</b> and <b>Google</b> are incorrectly tagged*
-  *all unknown numbers like <b>2013</b> and <b>2015</b> are incorrectly tagged*
-  *all unknown verbs like <b>contested</b> and <b>arriving</b> are incorrectly tagged*

> -  *overall accuracy obtained on test data set : <b>76.79%</b>*

### Solve Unknown Words Problem
***

### Method 1 : Combine Viterbi With Trigram State
***

> For an unknown word, use the following to compute a tag : 
-  calculate maximum likelihood estimate of a transition probability of :
     -  unigram tags
     -  bigram tags i.e. P(t2 | t1) = C(t1, t2) / C(t1)
     -  trigram tags i.e. P(t3 | t2, t1) = C(t1, t2, t3) / C(t1, t2)
-  if unknown word is the first word of a sentence, use **start probability**
-  if unknown word is the second word of a sentence, use **bigram probability**
-  if unknown word is present in any other position of a sentence, use **trigram probability**

### 1. Calculate Tag Sequences & Probabilities
***

#### Generate 'n' gram tag sequences

> -  Sample output when n = 1 
    -  `('.',), ('DET',), ('NOUN',), ('ADP',), ('ADJ',), ('NOUN',)`
-  Sample output when n = 2
   -  `('NOUN', 'ADJ'), ('ADJ', 'CONJ'), ('CONJ', 'ADP'), ('ADP', 'NUM')`
-  Sameple output when n = 3
   -  `('VERB', 'ADP', 'DET'), ('ADP', 'DET', 'ADJ'), ('DET', 'ADJ', 'NOUN')`

In [31]:
def generate_n_tag_seq(input_tags, n):
    tag_seq = [tuple(input_tags[i:(i + n)]) for i in range(len(input_tags) - n + 1)]
    return tag_seq

#### Generate 'n' gram tag counts

> -  Sample output for unigram tags 
    -  `('.',): 10611, ('DET',): 7047, ('NOUN',): 24722, ('ADP',): 8745`
-  Sample output for bigram tags
   -  `('X', 'ADP'): 915, ('.', 'VERB'): 950, ('VERB', 'DET'): 1723, ('NOUN', 'X'): 799`
-  Sameple output for trigram tags
   -  `('DET', 'NOUN', '.'): 862, ('NOUN', '.', 'DET'): 711, ('.', 'DET', 'NOUN'): 592`

In [32]:
def generate_tag_seq_counts(sent_tags):
    unigram_dict = defaultdict(int)
    bigram_dict = defaultdict(int)
    trigram_dict = defaultdict(int)

    unigram_tags = generate_n_tag_seq(sent_tags[2:], 1)
    bigram_tags = generate_n_tag_seq(sent_tags[1:], 2)
    trigram_tags = generate_n_tag_seq(sent_tags, 3)
    
    for unigram in unigram_tags:
        unigram_dict[unigram] += 1

    for bigram in bigram_tags:
        bigram_dict[bigram] += 1

    for trigram in trigram_tags:
        trigram_dict[trigram] += 1

    return unigram_dict, bigram_dict, trigram_dict

#### Generate 'n' gram tag probability

> Returns unigram, bigram and trigram tag sequence probabilities in 3 separate dictionary objects

In [33]:
def generate_tag_seq_prob(sent_tags, unigram_dict, bigram_dict, trigram_dict):
    unigram_prob_dict = defaultdict(int)
    bigram_prob_dict = defaultdict(int)
    trigram_prob_dict = defaultdict(int)

    unigram_total = sum(unigram_dict.values())

    unigram_prob_dict = {a: (unigram_dict[a] / unigram_total) for a in unigram_dict}

    bigram_prob_dict = {(a, b): (bigram_dict[(a, b)] / unigram_dict[(a, )])
                        for a, b in bigram_dict}
    
    trigram_prob_dict = {(a, b, c): (trigram_dict[(a, b, c)] / bigram_dict[(a, b)])
                         for a, b, c in trigram_dict}
    
    return unigram_prob_dict, bigram_prob_dict, trigram_prob_dict

#### Calculate 'n' gram tag probabilities
***

In [34]:
unigram_dict, bigram_dict, trigram_dict = generate_tag_seq_counts(all_tags)

unigram_prob_dict, bigram_prob_dict, trigram_prob_dict = generate_tag_seq_prob(
    all_tags, unigram_dict, bigram_dict, trigram_dict)

#### Calculate which tag has the maximum unigram probability

In [35]:
prob2tag = {v: k for k, v in unigram_prob_dict.items()}
max_prob = max(unigram_prob_dict.values())

tag_max_uni_prob = prob2tag[max_prob]
tag_max_uni_prob[0]

'NOUN'

#### Calculate most likely tag 't2' that a given tag 't1' will transition to

In [36]:
tag_big_pair = defaultdict()

for tag in unique_tags:
    bigram_prob_2_tag = {
        v: k
        for k, v in bigram_prob_dict.items() if k[0] == tag
    }
    max_prob = max(bigram_prob_2_tag.keys())

    max_bigram_prob_tup = bigram_prob_2_tag[max_prob]

    tag_big_pair[max_bigram_prob_tup[0]] = max_bigram_prob_tup[1]

tag_big_pair

defaultdict(None,
            {'.': 'NOUN',
             'ADJ': 'NOUN',
             'ADP': 'DET',
             'ADV': 'VERB',
             'CONJ': 'NOUN',
             'DET': 'NOUN',
             'NOUN': 'NOUN',
             'NUM': 'NOUN',
             'PRON': 'VERB',
             'PRT': 'VERB',
             'VERB': 'X',
             'X': 'VERB'})

In [37]:
def generate_t3_given_t2_and_t1(t1, t2):
    max_prob = 0

    for tag in unique_tags:
        try:
            v = trigram_prob_dict[(t1, t2, tag)]
            if v > max_prob:
                max_prob = v
                selected_tag = (t1, t2, tag)
        except KeyError:
            v = 0
    
    return selected_tag

### 2. Define Modified Viterbi With Trigram Tagger
***

In [38]:
def trigram_tagger(state, word):
    current_state_len = len(state)
    word_tag = ''
    
    if current_state_len == 0:
        # use unigram transition probbaility at the start of a sentence         
        word_tag = tag_max_uni_prob[0]
    elif current_state_len == 1:
        # use bigram transition probability for 2nd word of a sentence
        word_tag = tag_big_pair[state[-1]]
    else:
        # use tigram transition probability for all other words
        selected_tag = generate_t3_given_t2_and_t1(state[-2], state[-1])
        
        # special case for end of sentence
        # fallback to bigram transition probability
        if selected_tag[2] == '.' and word != '.':
            word_tag = tag_big_pair[state[-1]]
        else:
            word_tag = selected_tag[2]

    return word_tag

In [39]:
def Viterbi_With_Trigram_State(words, train_bag=train_tagged_words):
    state = []

    for key, word in enumerate(words):
        p = []
        for tag in unique_tags:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                if key == 0:
                    transition_p = df_tag.loc['.', tag]
                else:
                    transition_p = df_tag.loc[state[-1], tag]
            
            emission_p = word_given_tag(words[key], tag) / tag_count_dict[tag]
            state_probability = emission_p * transition_p
            p.append(state_probability)
        
        pmax = max(p)

        if pmax == 0:
            # invoke trigram tagger for unknown words where Viterbi fails to compute a tag
            try:
                last_end_of_sent = list(reversed(state)).index('.')
            except ValueError:
                # pass complete state in absence of a period
                last_end_of_sent = len(state)
                sent_state = state
                
            if last_end_of_sent == 0:
                sent_state = []
            else:
                sent_state = state[-last_end_of_sent:]

            # pass state of current sentence instead of complete state                     
            state_max = trigram_tagger(sent_state, word)
        else:
            state_max = unique_tags[p.index(pmax)]

        state.append(state_max)

    return list(zip(words, state))

### 3. Running Algorithm On Validation Dataset
***

In [40]:
%%time

tagged_seq_trigram = Viterbi_With_Trigram_State(validation_untagged_words)

Wall time: 35.3 s


### 4. Model Validation
***

In [41]:
check = [i for i, j in zip(tagged_seq_trigram, validation_run_base) if i == j]

accuracy = len(check)/len(tagged_seq_trigram)

print(accuracy)

0.9216867469879518


In [42]:
incorrect_tagged_cases_trigram = [[j] for i, j in enumerate(zip(tagged_seq_trigram, validation_run_base)) if j[0] != j[1]]

print(len(incorrect_tagged_cases_trigram))
incorrect_tagged_cases_trigram

13


[[(('sell', 'VERB'), ('sell', 'NOUN'))],
 [(('printers', 'ADP'), ('printers', 'NOUN'))],
 [(('start', 'NOUN'), ('start', 'VERB'))],
 [(('there', 'DET'), ('there', 'ADV'))],
 [(('assassinated', 'NOUN'), ('assassinated', 'VERB'))],
 [(('Arabian', 'ADP'), ('Arabian', 'NOUN'))],
 [(('pro-Iranian', 'NOUN'), ('pro-Iranian', 'ADJ'))],
 [(('Islamic', 'ADP'), ('Islamic', 'NOUN'))],
 [(('forthcoming', 'NOUN'), ('forthcoming', 'ADJ'))],
 [(('10-year', 'ADJ'), ('10-year', 'NUM'))],
 [(('yen-denominated', 'NOUN'), ('yen-denominated', 'ADJ'))],
 [(('about', 'ADP'), ('about', 'ADV'))],
 [(('redeeming', 'NOUN'), ('redeeming', 'VERB'))]]

> -  *Accuracy of Viterbi With Trigram Tagger - **low-90s** depending on validation dataset*
-  *Number of incorrect tagged words from validation set : **13** words*

### 5. Running Algorithm On Test Dataset
***

In [43]:
%%time

test_tagged_seq_tigram = Viterbi_With_Trigram_State(test_data_words)

Wall time: 36.7 s


In [44]:
print(test_tagged_seq_tigram)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'DET'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'),

#### Calculate Test Dataset Accuracy

In [45]:
calc_test_dataset_accuracy(test_tagged_seq_tigram)

total words - 181. correctly tagged words - 161. accuracy - 0.8895027624309392


In [46]:
test_unknown_tagged_words = [tup for tup in test_tagged_seq_tigram if tup[0] in test_data_unknown_words]

print(test_unknown_tagged_words)

[('Android', 'NOUN'), ('Google', 'DET'), ('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', 'NOUN'), ('2011', 'NOUN'), ('2013', 'NOUN'), ('Google', 'NOUN'), ('Twitter', 'NOUN'), ('2015', 'NOUN'), ('Google', 'NOUN'), ('Twitter', 'NOUN'), ('firehose', 'NOUN'), ('Twitter', 'NOUN'), ('online', 'NOUN'), ('interact', 'NOUN'), ('messages', 'NOUN'), ('tweets', 'DET'), ('domineering', 'NOUN'), ('personality', 'ADP'), ('2018', 'NOUN'), ('FIFA', 'ADP'), ('Cup', 'NOUN'), ('21st', 'NOUN'), ('FIFA', 'ADP'), ('Cup', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'NOUN'), ('Cup', 'NOUN'), ('trips', 'NOUN'), ('arriving', 'NOUN'), ('NASA', 'NOUN'), ('invited', 'NOUN'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN')]


> -  *proper nouns have been tagged correctly by this tagger : <b>Android, smartphones, Twitter</b>*
-  *unknown words tagged <b>incorrectly</b>*
 -  *numbers like 2011 & 2013*
 -  *verbs like domineering & invited*
 
> *overall tagging accuracy improved to <b>88.95%</b>, which includes both known and unknown words*

### Method 2 : Combine Viterbi With Rule Based Tagging
***

> For an unknown word, use a rule-based tagger instead of a trigram tagger

### 1. Define Modified Viterbi With Rule Based Tagger
***

In [47]:
def rule_based_tagger(word):

    # define regular expression patterns
    patterns = [
        (r'.*(ing|ed|es|ould)$', 'VERB'),
        (r'.*\'s$', 'NOUN'),
        (r'.*s$', 'NOUN'),
        (r'^[A-Z]+.*$', 'NOUN'),
        (r'^-?[0-9]+(.[0-9]+)?-?(.*)?$', 'NUM'),
        (r'.*', 'NOUN')  # nouns (default)
    ]

    # use nltk.RegexpTagger
    regexp_tagger = nltk.RegexpTagger(patterns)
    cal_tag = regexp_tagger.tag([word])
    
    return cal_tag[0][1]

In [50]:
def Viterbi_With_Regex(words, train_bag=train_tagged_words):
    state = []

    for key, word in enumerate(words):
        p = []
        for tag in unique_tags:
            if key == 0:
                transition_p = df_tag.loc['.', tag]
            else:
                transition_p = df_tag.loc[state[-1], tag]

            emission_p = word_given_tag(words[key], tag) / tag_count_dict[tag]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)

        if pmax == 0:
            # invoke rule-based tagger for unknown words where Viterbi fails to compute a tag
            state_max = rule_based_tagger(word)
        else:
            state_max = unique_tags[p.index(pmax)]

        state.append(state_max)

    return list(zip(words, state))

### 2. Running Algorithm On Validation Dataset
***

In [51]:
%%time

tagged_seq_regex = Viterbi_With_Regex(validation_untagged_words)

Wall time: 19.6 s


### 3. Model Validation
***

In [52]:
check = [i for i, j in zip(tagged_seq_regex, validation_run_base) if i == j]

accuracy = len(check)/len(tagged_seq_regex)

print(accuracy)

0.927710843373494


In [53]:
incorrect_tagged_cases_regex = [[j] for i, j in enumerate(zip(tagged_seq_regex, validation_run_base)) if j[0] != j[1]]

print(len(incorrect_tagged_cases_regex))
incorrect_tagged_cases_regex

12


[[(('sell', 'VERB'), ('sell', 'NOUN'))],
 [(('there', 'DET'), ('there', 'ADV'))],
 [(('pro-Iranian', 'NOUN'), ('pro-Iranian', 'ADJ'))],
 [(('slaying', 'VERB'), ('slaying', 'NOUN'))],
 [(('avenge', 'NOUN'), ('avenge', 'VERB'))],
 [(('beheading', 'VERB'), ('beheading', 'NOUN'))],
 [(('sweepstakes', 'VERB'), ('sweepstakes', 'NOUN'))],
 [(('forthcoming', 'VERB'), ('forthcoming', 'ADJ'))],
 [(('10-year', 'ADJ'), ('10-year', 'NUM'))],
 [(('yen-denominated', 'VERB'), ('yen-denominated', 'ADJ'))],
 [(('about', 'ADP'), ('about', 'ADV'))],
 [(('convert', 'NOUN'), ('convert', 'VERB'))]]

> -  *Accuracy of Viterbi With RegexTagger - **mid-90s** depending on validation dataset*
-  *Number of incorrect tagged words from validation set : **12** words*

### 4. Running Algorithm On Test Dataset
***

In [54]:
%%time

test_tagged_seq_regex = Viterbi_With_Regex(test_data_words)

Wall time: 23.8 s


In [56]:
print(test_tagged_seq_regex)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), (

#### Check tags assigned to unknown words
***

In [57]:
test_unknown_word_tags = [
    tag for tag in test_tagged_seq_regex if tag[0] in test_data_unknown_words
]

print(test_unknown_word_tags)

[('Android', 'NOUN'), ('Google', 'NOUN'), ('Android', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('smartphones', 'VERB'), ('2011', 'NUM'), ('2013', 'NUM'), ('Google', 'NOUN'), ('Twitter', 'NOUN'), ('2015', 'NUM'), ('Google', 'NOUN'), ('Twitter', 'NOUN'), ('firehose', 'NOUN'), ('Twitter', 'NOUN'), ('online', 'NOUN'), ('interact', 'NOUN'), ('messages', 'VERB'), ('tweets', 'NOUN'), ('domineering', 'VERB'), ('personality', 'NOUN'), ('2018', 'NUM'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('21st', 'NUM'), ('FIFA', 'NOUN'), ('Cup', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('Cup', 'NOUN'), ('trips', 'NOUN'), ('arriving', 'VERB'), ('NASA', 'NOUN'), ('invited', 'VERB'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN')]


#### Calculate Test Dataset Accuracy
***

In [58]:
calc_test_dataset_accuracy(test_tagged_seq_regex)

total words - 181. correctly tagged words - 170. accuracy - 0.9392265193370166


### 5. Unknowns Words Correctly Tagged By Method 2
***

> -  *almost all unknown words were tagged correctly with this technique*
     -  *<b>Android</b>, <b>Google</b> and <b>Twitter</b> correctly tagged as NOUN*
     -  *<b>domineering</b> and <b>invited</b> correctly tagged as VERB*
     -  *<b>2011</b>, <b>2013</b> and <b>2015</b> correctly tagged as NUMBER*
-  *unknown words tagged incorrectly : smartphones*

> *overall tagging accuracy improved further to <b>93.9%</b>, which includes both known and unknown words*

### Comparison
***

| no | word        | default | trigram | rule based | correct tag |
|:--:|-------------|:-------:|:-------:|:----------:|:-----------:|
|  1 |   Android   |    X    |   NOUN  |    NOUN    |     NOUN    |
|  2 |    Google   |    X    |   DET   |    NOUN    |     NOUN    |
|  3 | smartphones |    X    |   NOUN  |    VERB    |     NOUN    |
|  4 |     2011    |    X    |   NOUN  |   NUMBER   |    NUMBER   |
|  5 |     2015    |    X    |   NOUN  |   NUMBER   |    NUMBER   |
|  6 | domineering |    X    |   NOUN  |    VERB    |     VERB    |
|  7 |   invited   |    X    |   NOUN  |    VERB    |     VERB    |
|  8 |  contested  |    X    |   NOUN  |    VERB    |     VERB    |
|  9 |     21st    |    X    |   NOUN  |   NUMBER   |    NUMBER   |